In [7]:
# OpenAI API Key 정의
# **주의!** OpenAI API Key 는 외부에 노출하지 않도록 주의한다. 
# 4월 17일 이후에는 이전에 사용하던 사용자 키(User Key)보다 프로젝트별로 구분할 수 있는 프로젝트 키(Project Key)를 활용하는 것을 권장
# 일반적으로 시스템의 환경변수에 숨겨놓고 소스코드에는 노출하지 않지만, 
# 해당 과정은 파이썬 활용에 관한 내용이므로 예제에서는 생략한다.
# [모델 문서](https://platform.openai.com/docs/models)에는 사용가능한 모델의 이름이 나열되어있고, OpenAI에서 모델을 업데이트할 때마다 이곳에 명시하므로 자주 확인하면 좋다.

try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print(f"> 구글 코랩의 보안 비밀에서 OPENAI_API_KEY를 불러옵니다.")

except:
    OPENAI_API_KEY = "YOUR_OPENAI_PROJECT_KEY"
    print(f"> 로컬 환경에서 설정한 OPENAI_API_KEY를 사용합니다.")

OPENAI_MODEL = "gpt-4-turbo"

> 로컬 환경에서 설정한 OPENAI_API_KEY를 사용합니다.


In [2]:
# OpenAI API 패키지 import
try:
    import openai

except ModuleNotFoundError:
    # OpenAI API 패키지 설치
    !pip install openai pandas

finally:
    import openai

if "1.23.1" != openai.VERSION:
    print(f">> 예제는 2024-04-17 update 버전을 기준으로 작성하였습니다. 현재 버전 ({openai.VERSION})")

>> 예제는 2024-04-17 update 버전을 기준으로 작성하였습니다. 현재 버전 (1.30.1)


### [Assistants Tool] File Search

이 도구는 어시스턴트가 업로드한 파일들의 내용을 참고하여 사용자의 요청에 응답할 수 있는 기능 제공

⚠️"파일 검색(File Search)" 도구는 기존의 "지식 검색(Knowledge Retrieval)" 기능을 발전시켜 2024년 4월에 새롭게 변경되었다.

In [3]:
from pathlib import Path
import pandas as pd

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# 0. 데이터 파일 업로드 및 벡터 저장소 구성
report_files = [
    "datafiles/[LG전자]사업보고서(2024.03.18).pdf",
    "datafiles/[삼성전자]사업보고서(2024.03.12).pdf",
    "datafiles/[현대자동차]사업보고서(2024.03.13).pdf",
    "datafiles/alphabet-10-k-2023.pdf",
    "datafiles/amd-10-k-2023.pdf",
    "datafiles/apple-10-k-2023.pdf",
    "datafiles/nvidia-10-k-2023.pdf"
]

# 같은 파일을 중복 업로드하는 것을 방지하기 위해 목록에 없는 파일만 업로드한다.
filelist = [file.filename for file in client.files.list().data]

for filepath in report_files:
    if Path(filepath).name not in filelist:
        file = client.files.create(
            file=open(filepath, "rb"),
            purpose="assistants"
        )
        print(f"{filepath} : {file.id}")
    else:
        print(f"{filepath} : already uploaded.")


name_vector_store = "사업보고서 및 재무제표"

# 같은 벡터 저장소를 생성하지 않도록 벡터 저장소 목록을 확인한다.
vector_store_dict = {vs.id : vs.name for vs in client.beta.vector_stores.list()}

if name_vector_store not in vector_store_dict.values():
    file_list = client.files.list(purpose="assistants")
   
    df_files = pd.DataFrame([data.__dict__ for data in file_list.data])
    df_filtered = df_files[df_files['filename'].str.contains('사업보고서|10-k', case=False)]
    file_ids = df_filtered["id"].to_list()

    vector_store = client.beta.vector_stores.create(
        name=name_vector_store,
        file_ids=file_ids
    )
else:
    # 이미 같은 이름의 벡터 저장소가 있다면 첫 번째 벡터 저장소를 활용한다.
    print(f"{name_vector_store} : already created.")
    vsid = [key for key, value in vector_store_dict.items() if value == name_vector_store][0]
    vector_store = client.beta.vector_stores.retrieve(vector_store_id=vsid)
    print(f"{vector_store.id} : retrieved.")

datafiles/[LG전자]사업보고서(2024.03.18).pdf : already uploaded.
datafiles/[삼성전자]사업보고서(2024.03.12).pdf : already uploaded.
datafiles/[현대자동차]사업보고서(2024.03.13).pdf : already uploaded.
datafiles/alphabet-10-k-2023.pdf : already uploaded.
datafiles/amd-10-k-2023.pdf : already uploaded.
datafiles/apple-10-k-2023.pdf : already uploaded.
datafiles/nvidia-10-k-2023.pdf : already uploaded.
사업보고서 및 재무제표 : already created.
vs_afoIBC1a06w8uYFJ3wrgUeEf : retrieved.


In [4]:
# 1. 스레드 생성
thread = client.beta.threads.create()

# 2. 어시스턴트 생성; 파일 검색 도구 추가 
assistant = client.beta.assistants.create(
    name="재무 분석 전문가",
    instructions="""
        당신은 재무 분석 전문가입니다. 제공한 사업보고서 및 재무제표 정보를 바탕으로 질문에 답하시오.
        """,
    model=OPENAI_MODEL,
    tools=[{"type": "file_search"}],
    tool_resources={     
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    }
)

In [5]:
# 3. 사용자 메시지 생성
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="데이터의 각 회사가 발행한 주식의 수를 비교하시오."
)

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="각 회사의 2023년 주력 상품은 무엇이었나요?"
)

# 4. 런 생성 및 결과 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# 5. 런 상태 확인 및 런 스탭 내용 확인
if "completed" == run.status:
    run_steps = client.beta.threads.runs.steps.list(
        thread_id=thread.id,
        run_id=run.id,
        order="asc"
    )
    
    df_run_steps = pd.DataFrame([data.__dict__ for data in run_steps.data])
    for details in df_run_steps["step_details"]:
        if "tool_calls" == details.type:
            for tool_call in details.tool_calls:
                print(f"tool called : {tool_call.type}, {tool_call.id}")
                
        elif "message_creation" == details.type:
            msgid = details.message_creation.message_id
            msg = client.beta.threads.messages.retrieve(
                thread_id=thread.id,
                message_id=msgid
            )

            for content in msg.content:
                print(content.text.value)
                print("\nannotations >>\n")
                for annot in content.text.annotations:
                    file = client.files.retrieve(file_id=annot.file_citation.file_id)
                    print(annot.text, file.filename, annot.start_index, annot.end_index)
else:
    print(run.status)

tool called : file_search, call_sJUt8RveK3gCbU6kHhP29OH7
2023년 각 회사의 주력 상품은 다음과 같습니다.

1. **Apple Inc.**:
   - 1분기: iPad, iPad Pro, Apple TV 4K, Major League Soccer streaming service.
   - 2분기: MacBook Pro 14", MacBook Pro 16", Mac mini, HomePod.
   - 3분기: MacBook Air 15", Mac Studio, Mac Pro, Apple Vision Pro, iOS 및 macOS 업데이트.
   - 4분기: iPhone 15 시리즈, Apple Watch Series 9, Apple Watch Ultra 2【5:1†apple-10-k-2023.pdf】.

2. **삼성전자**:
   - 5G NR 및 FDD RU 제품, 듀얼 밴드 Massive MIMO MMU, 3.5GHz CBRS NR Strand Smallcell, LPDDR5X D램【5:3†삼성전자 사업보고서】.
   - 메모리카드 'PRO Ultimate', Automotive향 메모리 토탈 솔루션, 초저전력 차량용 UFS 3.1【5:7†삼성전자 사업보고서】.

3. **현대자동차**:
   - 아이오닉 5(NE), 베르나(BN7i)【5:5†현대자동차 사업보고서】.

4. **LG전자**:
   - OLED TV, QNED TV【5:8†LG전자 사업보고서】.

각 자료에서 확인된 정보를 바탕으로 이들 제품들이 각 회사의 2023년 주력 상품임을 알 수 있습니다.

annotations >>

【5:1†apple-10-k-2023.pdf】 apple-10-k-2023.pdf 340 365
【5:3†삼성전자 사업보고서】 [삼성전자]사업보고서(2024.03.12).pdf 472 488
【5:7†삼성전자 사업보고서】 [삼성전자]사업보고서(2024.03.12).pdf 557 573
【5:5†현대자동차 사업보고서】 [

In [6]:
# 6. 어시스턴트 삭제
res = client.beta.assistants.delete(
    assistant_id=assistant.id
)
print(f"assistant deleted: {res.deleted}")

# 7. 스레드 삭제
res = client.beta.threads.delete(
    thread_id=thread.id
)
print(f"thread deleted: {res.deleted}")


assistant deleted: True
thread deleted: True
